# Import packages

In [1]:
import pandas as pd
import numpy as np
import torch
from lightning import pytorch as pl

from chemprop import data
from chemprop import featurizers
from chemprop import models

# Change model input here

In [4]:
checkpoint_path = 'model.pt'

## Load model

In [5]:
mpnn = models.MPNN.load_from_checkpoint(checkpoint_path)
mpnn

KeyError: 'hyper_parameters'

# Change predict input here

In [6]:
test_path = '../tests/data/regression_test_smiles.csv'
smiles_column = 'smiles'

## Load test smiles

In [7]:
df_test = pd.read_csv(test_path)
df_test

,smiles
0,CCOP(=S)(OCC)SC(CCl)N2C(=O)c1ccccc1C2=O
1,Brc1ccc(Br)cc1
2,Cn2c(=O)on(c1ccc(Cl)c(Cl)c1)c2=O
3,Oc1ccc(cc1)c2ccccc2
4,CC1=C(CCCO1)C(=O)Nc2ccccc2
5,CCOC=C
6,CCC#C
7,COc1ncnc2nccnc12
8,CCCCC(C)(O)CC
9,Clc1ccc(Cl)cc1


## Get smiles

In [8]:
smis = df_test[smiles_column].tolist()
smis[:5]

['CCOP(=S)(OCC)SC(CCl)N2C(=O)c1ccccc1C2=O',
 'Brc1ccc(Br)cc1',
 'Cn2c(=O)on(c1ccc(Cl)c(Cl)c1)c2=O',
 'Oc1ccc(cc1)c2ccccc2',
 'CC1=C(CCCO1)C(=O)Nc2ccccc2']

## Get molecule datapoints

In [9]:
test_data = [data.MoleculeDatapoint.from_smi(smi) for smi in smis]

## Get molecule dataset

In [10]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()
test_dset = data.MoleculeDataset(test_data, featurizer=featurizer)
test_loader = data.MolGraphDataLoader(test_dset, shuffle=False)

TypeError: Can't instantiate abstract class MoleculeMolGraphFeaturizer with abstract method __call__

# Set up trainer

In [ ]:
with torch.inference_mode():
    trainer = pl.Trainer(
        logger=None,
        enable_progress_bar=True,
        accelerator="auto",
        devices=1
    )
    test_preds = trainer.predict(mpnn, test_loader)

In [ ]:
test_preds = np.concatenate(test_preds, axis=0)
df_test['pred'] = test_preds
df_test